# Convolutional Neural Networks (CNN)

커널 기반의 Convolution 연산을 사용하여 이미지 처리에 적합한 네트워크

![](https://miro.medium.com/max/1400/1*S7UzoZx2W2Wl75XKLjxPBg.jpeg)

[용어 자세한 설명](http://taewan.kim/post/cnn/)

## Convolution 연산

![](https://miro.medium.com/max/780/1*k_Kw7RPCfMaIY7USoAedqA.gif)

### 2D Convolution

![](https://miro.medium.com/max/738/1*tmKo_Dvlf3bHCMFx_PKZYA.gif)

## MaxPooling 연산

![](https://miro.medium.com/max/1400/1*vOxthD0FpBR6fJcpPxq6Hg.gif)

# 1. CNN을 구성하는 레이어

In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F

배치 크기 × 채널 × 높이(height) × 너비(widht)

- 배치 크기 : 1
- 채널 : 1 (그레이스케일 1, 컬러 3)
- 높이(픽셀) : 28
- 너비(픽셀) : 28

In [50]:
## Pooling example
input_example = torch.tensor([[[0, 1.0, 2], [3, 4, 5], [6, 7, 8]]])
print(input_example)
# max pooling
max_pooling_layer = nn.MaxPool2d(kernel_size=2, stride=1)
print(max_pooling_layer)
print(max_pooling_layer(input_example))
# average pooling
average_pooling_layer = torch.nn.AvgPool2d(kernel_size=2, stride=1)
print(average_pooling_layer)
print(average_pooling_layer(input_example))

tensor([[[0., 1., 2.],
         [3., 4., 5.],
         [6., 7., 8.]]])
MaxPool2d(kernel_size=2, stride=1, padding=0, dilation=1, ceil_mode=False)
tensor([[[4., 5.],
         [7., 8.]]])
AvgPool2d(kernel_size=2, stride=1, padding=0)
tensor([[[2., 3.],
         [5., 6.]]])


In [56]:
## Quiz!!
input_example = torch.tensor([[[12, 20, 30, 0.0], [20, 12, 2, 0], [0, 70, 5, 2], [8, 2, 90, 3]]])
print(input_example)

max_pooling_layer = nn.MaxPool2d(kernel_size=2, stride=2)
print('\nmax pooling:\n', max_pooling_layer(input_example))

average_pooling_layer = torch.nn.AvgPool2d(kernel_size=2, stride=2)
print('\naverage pooling:\n', average_pooling_layer(input_example))

tensor([[[12., 20., 30.,  0.],
         [20., 12.,  2.,  0.],
         [ 0., 70.,  5.,  2.],
         [ 8.,  2., 90.,  3.]]])

max pooling:
 tensor([[[20., 30.],
         [70., 90.]]])

average pooling:
 tensor([[[16.,  8.],
         [20., 25.]]])


## Input

In [2]:
inputs = torch.Tensor(1, 1, 32, 32)

inputs.shape

torch.Size([1, 1, 32, 32])

## 첫번째 Conv2D

In [3]:
conv1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, padding=1)

print(conv1)

Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))


## 두번째 Conv2D

In [4]:
conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1)

print(conv2)

Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))


## MaxPooling

In [5]:
pool = nn.MaxPool2d(kernel_size=2)

print(pool)

MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)


## 넣어보기

In [6]:
out = conv1(inputs)

print(out.shape)

torch.Size([1, 32, 32, 32])


In [7]:
out = pool(out)

print(out.shape)

torch.Size([1, 32, 16, 16])


In [8]:
out = conv2(out)

print(out.shape)

torch.Size([1, 64, 16, 16])


In [9]:
out = pool(out)

print(out.shape)

torch.Size([1, 64, 8, 8])


## Flatten

첫번째 차원인 배치 차원은 그대로 두고 나머지는 펼쳐라

In [10]:
out = out.view(out.size(0), -1) 

print(out.shape)

torch.Size([1, 4096])


## Dense (Fully connected)

In [11]:
fc = nn.Linear(4096, 10)

out = fc(out)

print(out.shape)
print(out)

torch.Size([1, 10])
tensor([[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]],
       grad_fn=<AddmmBackward0>)


# 2. CNN으로 MNIST 분류하기

Colab 에서 GPU 사용하기

런타임 - 런타임 유형 변경 - 하드웨어 가속기 - GPU - 저장

In [12]:
!nvidia-smi

Fri Apr  8 10:44:36 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P8    26W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [13]:
import torch
import torchvision.datasets as datasets
import torchvision.transforms as transforms
!pip install torchsummary
from torchsummary import summary

## GPU 사용 설정

In [14]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# 랜덤 시드 고정
torch.manual_seed(123)

# GPU 사용 가능일 경우 랜덤 시드 고정
if device == 'cuda':
    torch.cuda.manual_seed_all(123)

print(device)

cuda


## Hyperparameters

In [15]:
learning_rate = 0.001
training_epochs = 15
batch_size = 64

## Dataset & Dataloader

In [16]:
# transformation 정의하기
data_transform = transforms.Compose([
#             transforms.Resize((32, 32)),
            transforms.ToTensor(),
])

mnist_train = datasets.MNIST(
    root='MNIST_data/', # 다운로드 경로 지정
    train=True, # True를 지정하면 훈련 데이터로 다운로드
    transform=data_transform,
    download=True)

mnist_test = datasets.MNIST(
    root='MNIST_data/', # 다운로드 경로 지정
    train=False, # False를 지정하면 테스트 데이터로 다운로드
    transform=data_transform,
    download=True)

data_loader = torch.utils.data.DataLoader(
    dataset=mnist_train,
    batch_size=batch_size,
    shuffle=True,
    drop_last=True)

print('input sahep : %s, output : %s'%(mnist_train[0][0].shape, mnist_train[0][1]))

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting MNIST_data/MNIST/raw/train-images-idx3-ubyte.gz to MNIST_data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting MNIST_data/MNIST/raw/train-labels-idx1-ubyte.gz to MNIST_data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting MNIST_data/MNIST/raw/t10k-images-idx3-ubyte.gz to MNIST_data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting MNIST_data/MNIST/raw/t10k-labels-idx1-ubyte.gz to MNIST_data/MNIST/raw

input sahep : torch.Size([1, 28, 28]), output : 5


# 1_CNN 실습

### pytorch에서 모델 선언하는 3가지 방법

In [17]:
## define CNN model
from torch.autograd import Variable
class CNN1(nn.Module):
    def __init__(self): # input image = batch_size x 3 x 32 x 32
        super(CNN1, self).__init__()
        
        self.conv = nn.Conv2d(3, 64, kernel_size=3, padding=1)
        self.relu = nn.ReLU()
        self.maxpool = nn.MaxPool2d(2)

    def forward(self, x):
        out = self.conv(x)
        out = self.relu(out)
        out = self.maxpool(out)
        return out  # input image = batch_size x 3 x 16 x 16

    
class CNN2(nn.Module):
    def __init__(self): # input image = batch_size x 3 x 32 x 32
        super(CNN2, self).__init__()

        self.layer = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2))

    def forward(self, x):
        out = self.layer(x)
        return out  # input image = batch_size x 3 x 16 x 16      
      

class CNN3(nn.Module):
    def __init__(self): # input image = batch_size x 3 x 32 x 32
        super(CNN3, self).__init__()
        layer = []
        
        layer.append(nn.Conv2d(3, 64, kernel_size=3, padding=1))
        layer.append(nn.ReLU())
        layer.append(nn.MaxPool2d(2))
        
        self.layer = nn.Sequential(*layer)

    def forward(self, x):
        out = self.layer(x)
        return out  # input image = batch_size x 3 x 16 x 16

In [18]:
sample_image = Variable(torch.zeros(64, 3, 32, 32))

In [19]:
cnn = CNN1()
print(cnn)
output1 = cnn(sample_image)
print(output1.size())

CNN1(
  (conv): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu): ReLU()
  (maxpool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
)
torch.Size([64, 64, 16, 16])


In [20]:
cnn = CNN2()
print(cnn)
output2 = cnn(sample_image)
print(output2.size())

CNN2(
  (layer): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
)
torch.Size([64, 64, 16, 16])


In [21]:
cnn = CNN3()
print(cnn)
output3 = cnn(sample_image)
print(output3.size())

CNN3(
  (layer): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
)
torch.Size([64, 64, 16, 16])


# 네트워크 정의

In [22]:
class CNN_practice1(nn.Module):
    def __init__(self, n_classes=10): # input image = batch_size x 3 x 32 x 32
        super(CNN_practice1, self).__init__()
        
        # nn.Conv2d(in_channels, out_channels, kernel_size, stride=1, padding=0)
        
        self.conv1 = nn.Conv2d(3, 512, 3, 1, 1)  # 512 x 32 x 32
        self.conv2 = nn.Conv2d(512, 256, 3, 1, 1)   # 256 x 32 x 32
        self.conv3 = nn.Conv2d(256, 256, 3, 2, 1)  # 256 x 16 x 16
        self.conv4 = nn.Conv2d(256, 16, 3, 1, 1)  # 16 x 16 x 16 = 4096
    
        self.linear = nn.Linear(16*16*16, n_classes)  # 4096 -> 10
        
    def forward(self, x):
        out=self.conv1(x)
        
        out=self.conv2(out)
        
        out=self.conv3(out)
        
        out=self.conv4(out)
        
        out = out.contiguous().view(-1, 256*4*4)

        print(out.shape)
        
        out = self.linear(out)

        print(out.shape)        
        
        
        return out

model1=CNN_practice1(n_classes=10).to(device)
print(model1)
summary(model1, input_size=(3, 32, 32))

CNN_practice1(
  (conv1): Conv2d(3, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(512, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(256, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (conv4): Conv2d(256, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (linear): Linear(in_features=4096, out_features=10, bias=True)
)
torch.Size([2, 4096])
torch.Size([2, 10])
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1          [-1, 512, 32, 32]          14,336
            Conv2d-2          [-1, 256, 32, 32]       1,179,904
            Conv2d-3          [-1, 256, 16, 16]         590,080
            Conv2d-4           [-1, 16, 16, 16]          36,880
            Linear-5                   [-1, 10]          40,970
Total params: 1,862,170
Trainable params: 1,862,170
Non-trainable params: 0
--------------------------------------

In [23]:
class CNN_practice2(torch.nn.Module):
    def __init__(self):
        super(CNN_practice2, self).__init__()
        # 첫번째층
        # ImgIn shape=(?, 28, 28, 1)
        #    Conv     -> (?, 28, 28, 32)
        #    Pool     -> (?, 14, 14, 32)
        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))

        # 두번째층
        # ImgIn shape=(?, 14, 14, 32)
        #    Conv      ->(?, 14, 14, 64)
        #    Pool      ->(?, 7, 7, 64)
        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))

        # 전결합층 7x7x64 inputs -> 10 outputs
        self.fc = torch.nn.Linear(7 * 7 * 64, 10, bias=True)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0), -1)   # 전결합층을 위해서 Flatten
        out = self.fc(out)
        return out

model2 = CNN_practice2().to(device)
print(model2)
summary(model2, input_size=(1, 28, 28))

CNN_practice2(
  (layer1): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc): Linear(in_features=3136, out_features=10, bias=True)
)
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 28, 28]             320
              ReLU-2           [-1, 32, 28, 28]               0
         MaxPool2d-3           [-1, 32, 14, 14]               0
            Conv2d-4           [-1, 64, 14, 14]          18,496
              ReLU-5           [-1, 64, 14, 14]               0
         MaxPool2d-6             [-1, 64, 7, 7]               0
    

In [24]:
## LeNet
# https://deep-learning-study.tistory.com/368
class LeNet5(nn.Module):
    def __init__(self, n_classes):
        super(LeNet5, self).__init__()
        
        self.softmax = nn.Softmax(dim=1)
        
        self.feature_extractor = nn.Sequential(            
            nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5, stride=1),
            nn.Tanh(),
            nn.AvgPool2d(kernel_size=2),
            nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5, stride=1),
            nn.Tanh(),
            nn.AvgPool2d(kernel_size=2),
            nn.Conv2d(in_channels=16, out_channels=120, kernel_size=5, stride=1),
            nn.Tanh()
        )

        self.classifier = nn.Sequential(
            nn.Linear(in_features=120, out_features=84),
            nn.Tanh(),
            nn.Linear(in_features=84, out_features=n_classes),
        )


    def forward(self, x):
        x = self.feature_extractor(x)
        x = torch.flatten(x, 1)
        logits = self.classifier(x)
        probs = self.softmax(logits)
        return logits, probs

N_CLASSES = 10
model3 = LeNet5(N_CLASSES).to(device)
print(model3)
summary(model3, input_size=(1, 32, 32))

LeNet5(
  (softmax): Softmax(dim=1)
  (feature_extractor): Sequential(
    (0): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
    (1): Tanh()
    (2): AvgPool2d(kernel_size=2, stride=2, padding=0)
    (3): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
    (4): Tanh()
    (5): AvgPool2d(kernel_size=2, stride=2, padding=0)
    (6): Conv2d(16, 120, kernel_size=(5, 5), stride=(1, 1))
    (7): Tanh()
  )
  (classifier): Sequential(
    (0): Linear(in_features=120, out_features=84, bias=True)
    (1): Tanh()
    (2): Linear(in_features=84, out_features=10, bias=True)
  )
)
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 6, 28, 28]             156
              Tanh-2            [-1, 6, 28, 28]               0
         AvgPool2d-3            [-1, 6, 14, 14]               0
            Conv2d-4           [-1, 16, 10, 10]           2,416
              Tanh-5           [-1, 

In [25]:
## AlexNet
# https://github.com/pytorch/vision/blob/main/torchvision/models/alexnet.py
class AlexNet(nn.Module):
    def __init__(self, num_classes: int = 1000, dropout: float = 0.5) -> None:
        super().__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=11, stride=4, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(64, 192, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
        )
        self.avgpool = nn.AdaptiveAvgPool2d((6, 6))
        self.classifier = nn.Sequential(
            nn.Dropout(p=dropout),
            nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(p=dropout),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, num_classes),
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x
    
model4 = AlexNet(num_classes=1000).to(device)
print(model4)
summary(model4, input_size=(3, 227, 227))  # ImageNet의 입력사이즈 3X227X226

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [72]:
## Residual Connection
class ResBlock(nn.Module):
    def __init__(self, ni):
        super(ResBlock, self).__init__()
        self.conv1 = nn.Conv2d(ni, ni, 1)
        self.conv2 = nn.Conv2d(ni, ni, 3, 1, 1)
        self.classifier = nn.Linear(ni*24*24, 10)

    def forward(self,x):
        residual = x
        out = F.relu(self.conv1(x))
        out = F.relu(self.conv2(out))
        
        out += residual
        
        out = out.view(out.size(0),-1)
        return self.classifier(out)

block = ResBlock(3)
x = torch.randn(1, 3, 24, 24)
output = block(x)
print(output.shape)

torch.Size([1, 10])


## 로스, 옵티마이저 정의

In [26]:
model = model2

In [27]:
criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [28]:
for epoch in range(training_epochs):
    avg_loss = 0

    for x_train, y_train in data_loader:
        x_train = x_train.to(device)
        y_train = y_train.to(device)

        y_pred = model(x_train)

        optimizer.zero_grad()
        loss = criterion(y_pred, y_train)
        loss.backward()
        optimizer.step()

        avg_loss += loss / len(data_loader) # loss / 배치의 갯수

    print('[Epoch: {:>4}] loss = {:>.9}'.format(epoch + 1, avg_loss))

[Epoch:    1] loss = 0.179870427
[Epoch:    2] loss = 0.055393476
[Epoch:    3] loss = 0.0411735997
[Epoch:    4] loss = 0.0330234393
[Epoch:    5] loss = 0.0270446781
[Epoch:    6] loss = 0.0224145446
[Epoch:    7] loss = 0.0180260204
[Epoch:    8] loss = 0.0151498085
[Epoch:    9] loss = 0.0125806704
[Epoch:   10] loss = 0.0104894089
[Epoch:   11] loss = 0.00882262923
[Epoch:   12] loss = 0.0080840718
[Epoch:   13] loss = 0.00627558259
[Epoch:   14] loss = 0.00570008019
[Epoch:   15] loss = 0.00586899649


In [29]:
with torch.no_grad(): # Gradient를 업데이트하지 않는다
    x_test = mnist_test.data.view(len(mnist_test), 1, 28, 28).float().to(device)
    y_test = mnist_test.targets.to(device)

    y_pred = model(x_test)
    correct_prediction = torch.argmax(y_pred, 1) == y_test
    accuracy = correct_prediction.float().mean()

    print('Accuracy:', accuracy.item())

Accuracy: 0.9840999841690063


# (숙제) 정확도 99%로 만들어보자!